Task Goal
Calculate the probability of being born in Canada in a particular year, using World Bank data on:

- Birth Rate (SP.DYN.CBRT.IN) → births per 1,000 people

- Total Population (SP.POP.TOTL) → total country population per year

births = (birth_rate / 1000) * population
probability = births_canada / births_world

#### Fetch Data, world population and birth rate

In [ ]:
# fetch birth rate for all countries
import wbgapi as wb
import pandas as pd
import pprint

# population fetch
time=range(2019,2024)
economy='all'
series_totl_pop='SP.POP.TOTL'


totl_pop_country_wise_dump = wb.data.DataFrame(series_totl_pop,economy=economy,time=time,labels=True).reset_index()
print(totl_pop_country_wise_dump.head())
# pprint.pprint(totl_pop_country_wise_dump.head())

  economy                Country      YR2019      YR2020      YR2021  \
0     ZWE               Zimbabwe  15271368.0  15526888.0  15797210.0   
1     ZMB                 Zambia  18513839.0  19059395.0  19603607.0   
2     YEM            Yemen, Rep.  35111408.0  36134863.0  37140230.0   
3     PSE     West Bank and Gaza   4685306.0   4803269.0   4922749.0   
4     VIR  Virgin Islands (U.S.)    106669.0    106290.0    105870.0   

       YR2022      YR2023  
0  16069056.0  16340822.0  
1  20152938.0  20723965.0  
2  38222876.0  39390799.0  
3   5043612.0   5165775.0  
4    105413.0    104917.0  


In [12]:
#condition population data
totl_pop_country_wise = totl_pop_country_wise_dump.melt(
    id_vars=['economy','Country'],
    var_name='year',
    value_name='population'
)
totl_pop_country_wise.columns=['code','country','year','population']
totl_pop_country_wise['year']=totl_pop_country_wise['year'].str.replace('YR','').astype(int)
print(totl_pop_country_wise.head())
print(len(totl_pop_country_wise))

totl_pop_by_year = totl_pop_country_wise.groupby('year')['population'].sum().reset_index()
print(totl_pop_by_year.head())

  code                country  year  population
0  ZWE               Zimbabwe  2019  15271368.0
1  ZMB                 Zambia  2019  18513839.0
2  YEM            Yemen, Rep.  2019  35111408.0
3  PSE     West Bank and Gaza  2019   4685306.0
4  VIR  Virgin Islands (U.S.)  2019    106669.0
1330
   year    population
0  2019  8.354924e+10
1  2020  8.448114e+10
2  2021  8.531826e+10
3  2022  8.615821e+10
4  2023  8.703543e+10


In [16]:
# fetch birth rate data
time=range(2019,2024)
economy='all'
series2='SP.DYN.CBRT.IN'

birth_rate_country_wise_dump = wb.data.DataFrame(series=series2,economy=economy,time=time,labels=True).reset_index()
print(birth_rate_country_wise_dump.head())

  economy                Country  YR2019  YR2020  YR2021  YR2022  YR2023
0     ZWE               Zimbabwe  31.121  30.988  30.932  30.882  30.410
1     ZMB                 Zambia  35.006  34.408  33.930  33.542  33.081
2     YEM            Yemen, Rep.  35.947  35.895  35.668  35.407  35.209
3     PSE     West Bank and Gaza  29.708  28.877  28.225  27.575  27.062
4     VIR  Virgin Islands (U.S.)  12.300  12.100  11.800  11.600  11.400


There is a data of INX having NaN for birthrate, same is replaced with mean() of the entire column in order to keep the birthrate newtral.

In [ ]:
# condition population data
birth_rate_country_wise = birth_rate_country_wise_dump.melt(
    id_vars=['economy','Country'],
    var_name='year',
    value_name='birthrate'
)
birth_rate_country_wise.columns=['code','country','year','birthrate']
birth_rate_country_wise['year']=birth_rate_country_wise['year'].str.replace('YR','').astype(int)
print(birth_rate_country_wise.head())
print(len(birth_rate_country_wise))
na_count = birth_rate_country_wise.isna().sum().reset_index()
print(na_count)
print(birth_rate_country_wise[birth_rate_country_wise.isna().any(axis=1)])
if(birth_rate_country_wise[birth_rate_country_wise['code']=='INX']):
    birth_rate_country_wise['birthrate']= birth_rate_country_wise['birthrate'].fillna(birth_rate_country_wise['birthrate'].mean())

  code                country  year  birthrate
0  ZWE               Zimbabwe  2019     31.121
1  ZMB                 Zambia  2019     35.006
2  YEM            Yemen, Rep.  2019     35.947
3  PSE     West Bank and Gaza  2019     29.708
4  VIR  Virgin Islands (U.S.)  2019     12.300
1330
       index  0
0       code  0
1    country  0
2       year  0
3  birthrate  0
Empty DataFrame
Columns: [code, country, year, birthrate]
Index: []


Store data in sqlite database for futher calculations

In [ ]:
#create data base, and create tables to store population and birthrate data
import sqlite3

conn = sqlite3.connect('mapleroot.db',isolation_level=None)
totl_pop_by_year.to_sql('tbl_totl_pop',conn,if_exists='replace',index=False)
birth_rate_country_wise.to_sql('tbl_br_cnt',conn,if_exists='replace',index=False)
print(conn.execute('SELECT * FROM sqlite_schema WHERE type="table"').fetchall())
print(conn.execute('PRAGMA TABLE_INFO("tbl_totl_pop")').fetchall())
print(conn.execute('PRAGMA TABLE_INFO("tbl_br_cnt")').fetchall())
conn.close()

[('table', 'tbl_totl_pop', 'tbl_totl_pop', 2, 'CREATE TABLE "tbl_totl_pop" (\n"year" INTEGER,\n  "population" REAL\n)'), ('table', 'tbl_br_cnt', 'tbl_br_cnt', 3, 'CREATE TABLE "tbl_br_cnt" (\n"code" TEXT,\n  "country" TEXT,\n  "year" INTEGER,\n  "birthrate" REAL\n)')]
[(0, 'year', 'INTEGER', 0, None, 0), (1, 'population', 'REAL', 0, None, 0)]
[(0, 'code', 'TEXT', 0, None, 0), (1, 'country', 'TEXT', 0, None, 0), (2, 'year', 'INTEGER', 0, None, 0), (3, 'birthrate', 'REAL', 0, None, 0)]
